In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2 # for preprocessing

# For preprocessing and model creation
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

# These libraries are used to load the dataset
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Python Image Library used for image preprocessing
from PIL import Image,ImageOps

# is an optimization algorithm commonly used for training deep learning models
from torch.optim import Adam

# (PyTorch Image Models) Used for creating the model
!pip install timm
from timm.models.vision_transformer import VisionTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00


In [3]:
# called to free up currently occupies GPU memory
torch.cuda.empty_cache()

# is used to determine and set the CPU or GPU for tensor computations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
############## Dataset Preparation

transform_train = transforms.Compose([
    transforms.Resize((384, 384), antialias=True),  ## Resize input images
    transforms.RandomHorizontalFlip(), ## randomly rotates the training image horizontaly
    # transforms.Grayscale(),  # Convert images to grayscale
    transforms.ToTensor(), ## converts the image to a tensor
    # transforms.Normalize((0.406), (0.406))  # Normalize images
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ## This is where the normalizing is done
])

transform_test = transforms.Compose([
    transforms.Resize((384, 384), antialias=True),
    # transforms.Grayscale(),  # Convert images to grayscale
    transforms.ToTensor(),
    # transforms.Normalize((0.485), (0.485)),  # Normalize images
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root='/content/drive/MyDrive/BikeProject/Tyre Component/train', transform=transform_train)
test_dataset = ImageFolder(root='/content/drive/MyDrive/BikeProject/Tyre Component/test', transform=transform_test)

## Data loaders are created here
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [5]:
# Apply image filtering

def apply_filter(image):
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)  # Apply Gaussian smoothing
    return blurred_image



In [6]:
# Apply histogram equalization

def apply_equalization(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply histogram equalization
    equalized_image = cv2.equalizeHist(gray_image)
    # Convert image back to BGR if needed
    equalized_image = cv2.cvtColor(equalized_image, cv2.COLOR_GRAY2BGR)
    return equalized_image



In [7]:
# Model Initialization

num_classes = len(train_dataset.classes)
model = VisionTransformer(img_size=384, patch_size=16, in_chans=3, num_classes=num_classes)

In [8]:
############### Model Training

model.to(device)
criterion = nn.CrossEntropyLoss()

# I have used Adam  (Adaptive Moment Estimation) optimizer here
optimizer = Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        outputs = nn.functional.softmax(outputs, dim=1)  # Apply softmax activation to the model. Faster than Sigmoid
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/10], Loss: 1.2223
Epoch [2/10], Loss: 1.2146
Epoch [3/10], Loss: 1.2146
Epoch [4/10], Loss: 1.1917
Epoch [5/10], Loss: 1.2145
Epoch [6/10], Loss: 1.2057
Epoch [7/10], Loss: 1.1932
Epoch [8/10], Loss: 1.1932
Epoch [9/10], Loss: 1.2161
Epoch [10/10], Loss: 1.1932


In [9]:
# Evaluation
model.eval()
correct = 0
total = 0

predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        outputs = nn.functional.softmax(outputs, dim=1)  # Apply softmax activation to the model'
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        predictions.extend(predicted.cpu().numpy())

accuracy = 100 * correct / total
print(num_classes) # print number of classes
print(f"Accuracy on the test set: {accuracy:.2f}%") ## printing accuracy


3
Accuracy on the test set: 31.75%


In [10]:
# Prediction Function

def classify_image(image_path):
    image = Image.open(image_path)  # Load and convert image to grayscale
    image = transform_test(image).unsqueeze(0)
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)
        outputs = nn.functional.softmax(output, dim=1)  # Apply softmax activation to the model'
        _, predicted = torch.max(output, 1)
        return predicted.item()

# Define the class label to name mapping
class_mapping = {
    0: "good",
    1: "moderate",
    2: "bad",
}

In [11]:
# Example usage
image_path = '/content/drive/MyDrive/BikeProject/Tyre Component/valid/bad/10.jpg'
predicted_class = classify_image(image_path)
print(f"Predicted class: {predicted_class}")
print(f"Predicted class: {class_mapping.get(predicted_class)}")

Predicted class: 0
Predicted class: good


In [12]:
# Example usage
image_path = '/content/drive/MyDrive/BikeProject/Tyre Component/valid/good/20230517_151919.jpg'
predicted_class = classify_image(image_path)
print(f"Predicted class: {predicted_class}")
print(f"Predicted class: {class_mapping.get(predicted_class)}")

Predicted class: 0
Predicted class: good


In [13]:
# Example usage
image_path = '/content/drive/MyDrive/BikeProject/Tyre Component/valid/moderate/20230517_153737.jpg'
predicted_class = classify_image(image_path)
print(f"Predicted class: {predicted_class}")
print(f"Predicted class: {class_mapping.get(predicted_class)}")

Predicted class: 0
Predicted class: good


In [ ]:
# Save the trained model
# torch.save(model.state_dict(), "vision_transformer_model.pth")